# Colocalization analyses
## 2025-09-02

In [ ]:
fig.size <- function(height, width, res = 400) {
    options(repr.plot.height = height, repr.plot.width = width, repr.plot.res = res)
}
suppressPackageStartupMessages({
    library(metafor)
    library(data.table)
    library(sf)
    library(purrr)
    library(ggplot2)
    library(ggthemes)
    library(stringr)
    library(scales)
    library(circlize)
    library(tidyr)
    library(glue)
    library(ComplexHeatmap)
    library(scales)
    library(presto)
    library(lme4)
    library(singlecellmethods)
    library(viridis)
    library(furrr)
    library(future)
    plan(multicore)
    library(dplyr)
    library(patchwork)
    
    
})


# Functions for colocalization analysis

In [ ]:
# Load necessary libraries
# Ensure you have these installed: install.packages(c("data.table", "Matrix", "spatula", "presto", "furrr", "dplyr", "metafor"))
library(data.table)
library(Matrix)
library(spatula)
library(presto)
library(furrr)
library(dplyr)
library(metafor)

#' Calculate Cell Type Spatial Enrichment
#'
#' This function analyzes spatial relationships between cell types on a per-sample basis.
#' For each "anchor" cell, it counts the number of "neighbor" cells within a defined
#' neighborhood and uses a mixed-effects model (via presto) to determine if certain
#' neighbor types are enriched around certain anchor types.
#'
#' @param cells A data.table or data.frame containing cell data. Must include columns
#'   for 'SampleID', spatial coordinates 'X' and 'Y', and cell type 'type_lvl3'.
#' @param anchor_types A character vector specifying the cell types to be treated as
#'   the central "anchor" cells in the analysis.
#' @param neighbor_types A character vector specifying the cell types to be counted
#'   as "neighbors".
#' @param max_dist A numeric value for the maximum distance (in the same units as X/Y
#'   coordinates) to consider two cells neighbors. Connections longer than this are
#'   pruned. Defaults to 30.
#' @param nsteps An integer specifying the number of steps to expand the neighborhood.
#'   nsteps = 1 means immediate neighbors, nsteps = 2 includes neighbors of neighbors, etc.
#'   Defaults to 3.
#' @param use_multicore A logical value indicating whether to process samples in parallel
#'   using `furrr` with the `multicore` plan. Defaults to TRUE.
#'
#' @return A data.table containing the enrichment analysis results for each sample,
#'   with columns for anchor type, neighbor type, log-fold change, standard error, etc.
#'
calculate_spatial_enrichment <- function(cells,
                                         anchor_types,
                                         neighbor_types,
                                         max_dist = 30,
                                         nsteps = 3,
                                         use_multicore = TRUE) {
  
  # Set up the parallel processing plan based on the user's choice.
  # plan(sequential) runs tasks one by one.
  # plan(multicore) runs tasks in parallel on the same machine.
  if (use_multicore) {
    plan(multicore)
  } else {
    plan(sequential)
  }
  
  # Process each sample independently.
  # We split the main 'cells' data.table by 'SampleID'.
  # 'future_imap' then iterates over each sample's data chunk.
  res_all <- cells %>%
    split(.$SampleID) %>%
    future_imap(function(.cells, .name) {
      
      # Print the name of the sample being processed for progress tracking.
      message("Processing sample: ", .name)
      
      # --- 1. Define Spatial Neighbors ---
      
      # Extract X and Y coordinates into a matrix.
      coords <- as.matrix(.cells[, .(X, Y)])

      # Center the coordinates without changing their scale
      #coords <- scale(coords, center = TRUE, scale = FALSE)
      # 'getSpatialNeighbors' creates an adjacency matrix where non-zero entries
      # indicate neighboring cells and the values are the distances.
      adj <- spatula::getSpatialNeighbors(coords, return_weights = TRUE)
      
      # Prune long-distance connections: set distances greater than 'max_dist' to 0.
      adj@x[adj@x > max_dist] <- 0
      
      # 'drop0' removes the explicit zero entries, making the matrix sparse and efficient.
      adj <- Matrix::drop0(adj)
      
      # Convert the adjacency matrix to be unweighted (binary: 1 for neighbor, 0 otherwise).
      adj@x <- rep(1, length(adj@x))
      
      # Sanity check: ensure no cell is counted as its own neighbor.
      stopifnot(all(diag(adj) == 0))
      
      # --- 2. Get Neighborhood Compositions ---
      
      # Create a sparse matrix where rows are cells and columns are cell types.
      # This represents the "0-step" neighbors (the cell itself).
      counts <- Matrix::sparse.model.matrix(~ 0 + type_lvl3, .cells)
      
      # Expand the neighborhood by 'nsteps'.
      # Matrix multiplication aggregates neighbor counts at each step.
      for (iter in seq_len(nsteps)) {
        counts <- adj %*% counts
      }
      
      # Clean up column names for clarity.
      colnames(counts) <- gsub('type_lvl3', '', colnames(counts))
      
      # --- 3. Filter Cells and Neighbors for Modeling ---
      
      # Only consider specified neighbor types.
      valid_neighbor_types <- intersect(neighbor_types, colnames(counts))
      counts <- counts[, valid_neighbor_types, drop = FALSE]
      
      # Calculate the total number of relevant neighbors for each cell and log-transform it.
      # This will be used as an offset in the model to account for neighborhood density.
      .cells$log_n_neighbors <- log(rowSums(counts))
      
      # Filter out cells that have zero neighbors (where log(0) is -Inf).
      i <- which(!is.infinite(.cells$log_n_neighbors))
      M <- .cells[i, ]
      counts <- counts[i, , drop = FALSE]
      
      # Limit the analysis to the specified anchor cell types.
      i <- which(M$type_lvl3 %in% anchor_types)
      M <- M[i, ]
      counts <- counts[i, , drop = FALSE]
      
      # --- 4. Run Statistical Model (presto) ---
      
      # Suppress warnings that may arise during model fitting.
      suppressWarnings({
        # 'presto.presto' fits a fast generalized linear mixed model.
        # Formula: y ~ 1 + (1|type_lvl3) + offset(log_n_neighbors)
        # - y: The count of a specific neighbor type (handled internally by presto).
        # - (1|type_lvl3): A random intercept for each anchor cell type.
        # - offset(log_n_neighbors): Accounts for the total number of neighbors.
        presto_res <- presto::presto.presto(
          y ~ 1 + (1 | type_lvl3) + offset(log_n_neighbors),
          M,
          t(counts), # Counts matrix needs to be transposed for presto.
          'log_n_neighbors',
          effects_cov = c('type_lvl3'),
          nsim = 1000,
          ncore = 1, # Parallelism is handled by furrr at the sample level.
          family = 'poisson',
          min_sigma = .05,
          verbose = FALSE
        )
      })
      
      # --- 5. Calculate and Format Contrasts ---
      
      # Create a contrast matrix to compare anchor types.
      contrasts_mat <- make_contrast.presto(
        object = presto_res,
        var_contrast = 'type_lvl3'
      )
      contrasts_mat[is.na(contrasts_mat)] <- 0 # Replace NAs with 0.
      
      # Calculate the effects based on the contrast matrix.
      eff <- contrasts.presto(presto_res, contrasts_mat, one_tailed = FALSE) %>%
        dplyr::rename(anchor = contrast, neighbor = feature, logFC = beta, logSE = sigma) %>%
        dplyr::mutate(
          # Convert from natural log (ln) to log base 2 for easier interpretation.
          logFC = logFC / log(2),
          logSE = logSE / log(2)
        ) %>%
        data.table()
      
      return(eff)
      
    }, .options = furrr::furrr_options(seed = TRUE)) %>% # Use a seed for reproducibility.
    bind_rows(.id = 'SampleID') # Combine results from all samples into one data.table.
  
  return(res_all)
}


#' Summarize Enrichment Results with a Meta-Analysis
#'
#' This function takes the per-sample enrichment results and combines them using a
#' random-effects model with inverse variance weighting to produce a single,
#' summarized result for each anchor-neighbor pair.
#'
#' @param enrichment_results A data.table, typically the output from
#'   `calculate_spatial_enrichment`.
#' @param n_total The total number of samples, used in the REML prior calculation.
#' @param c_prior The prior value for the REML calculation.
#'
#' @return A final, summarized data.table with meta-analyzed logFC, p-values,
#'   and adjusted p-values for each anchor-neighbor interaction.
#'
summarize_enrichment_results <- function(enrichment_results, n_total = 8, c_prior = 0.1) {
  require(data.table)
  # This step groups the data by each anchor-neighbor pair and applies the
  # meta-analysis function to combine logFC and logSE values across all samples.
  res <- enrichment_results[, 
    combine_with_REML_prior(.SD$logFC, .SD$logSE, n_total = n_total, c_prior = c_prior),
    by = .(anchor, neighbor)
  ][
    order(pvalue) # Order results by p-value.
  ][
    !is.na(logFC) # Remove rows where logFC is NA.
  ][
    , padj := p.adjust(pvalue, 'BH') # Calculate Benjamini-Hochberg adjusted p-values.
  ]
  
  return(res)
}


# --- Meta-Analysis Helper Functions ---

#' Estimate Heterogeneity (Tau) via REML
#'
#' This function estimates the between-dataset standard deviation (tau), a measure of
#' heterogeneity, from observed effect sizes and their standard errors. It uses the
#' `rma.uni` function from the `metafor` package.
#'
#' @param est A numeric vector of effect size estimates (e.g., log-fold changes).
#' @param se A numeric vector of standard errors corresponding to the estimates.
#' @param method A character string specifying the method for estimating tau-squared.
#'   Defaults to "REML" (Restricted Maximum-Likelihood).
#' @return A numeric value representing the estimated between-study standard deviation (tau).
#'
tau_reml <- function(est, se, method = "REML") {
  # Ensure the estimates and standard errors are of the same length.
  stopifnot(length(est) == length(se))
  
  # Fit a random-effects meta-analysis model using the metafor package.
  # 'yi' are the effect sizes, and 'sei' are their standard errors.
  fit <- metafor::rma.uni(yi = est, sei = se, method = method)
  
  # Extract tau^2 (the variance) and take the square root to get tau (the standard deviation).
  as.numeric(sqrt(fit$tau2))
}

#' Combine Estimates with a Zero-Centered Prior for Missing Data
#'
#' This function performs a random-effects meta-analysis that incorporates a
#' zero-centered prior for "missing" studies. This is a form of regularization that
#' shrinks the overall estimate towards zero when data is sparse (i.e., when an
#' effect is observed in only a few out of many possible datasets).
#'
#' @param est A numeric vector of the *observed* effect size estimates.
#' @param se A numeric vector of the *observed* standard errors.
#' @param n_total An integer for the total number of datasets that *could* have
#'   contributed data (observed + missing).
#' @param tau_hat A numeric value for the estimated between-study heterogeneity (tau),
#'   typically from `tau_reml`.
#' @param c_prior A numeric scaling factor for the prior's standard deviation. The
#'   prior's SD is calculated as `tau0 = c_prior * tau_hat`. Defaults to 1.
#' @param se_floor An optional numeric value to prevent a too-strong (too narrow) prior
#'   when `tau_hat` is close to zero. The prior's SD will be `max(tau0, se_floor)`.
#' @return A data.table containing the meta-analyzed logFC, logSE, z-score, p-value,
#'   and estimates of tau.
#'
meta_with_zero_prior_RE <- function(est, se, n_total, tau_hat,
                                      c_prior = 1, se_floor = NULL) {
  # Perform sanity checks on the inputs.
  stopifnot(length(est) == length(se), n_total >= length(est))
  
  # Calculate the number of observed and missing studies.
  k_obs  <- length(est)
  k_miss <- n_total - k_obs
  
  # Calculate random-effects weights for the observed studies.
  # The weight for each study is the inverse of its variance (se^2 + tau_hat^2).
  w_obs <- 1 / (se^2 + tau_hat^2)
  w_obs_sum <- sum(w_obs)
  
  # Define the prior's standard deviation (tau0). The prior assumes missing studies
  # have an effect size of 0 with a standard deviation of tau0.
  tau0 <- c_prior * tau_hat
  
  # If an se_floor is provided, ensure the prior is never stronger (more certain)
  # than a typical observed SE. This prevents overly strong shrinkage if tau_hat is near zero.
  if (!is.null(se_floor)) {
    tau0 <- max(tau0, se_floor)
  }
  
  # Calculate the weight for each "missing" pseudo-study.
  w0 <- if (k_miss > 0) 1 / (tau0^2) else 0
  
  # Calculate the pooled estimate and its standard error.
  # This is a weighted average of the observed effects and the k_miss pseudo-studies at effect 0.
  w_tot <- w_obs_sum + k_miss * w0
  mu    <- if (w_tot > 0) sum(w_obs * est) / w_tot else NA_real_
  se_mu <- if (w_tot > 0) sqrt(1 / w_tot)       else NA_real_
  
  # Calculate z-score and two-tailed p-value.
  z     <- mu / se_mu
  p     <- 2 * (1 - pnorm(abs(z)))
  
  # Return the results in a data.table.
  data.table(
    logFC = mu, logSE = se_mu, zscore = z, pvalue = p,
    tau_hat = tau_hat, tau0 = tau0
  )
}

#' Wrapper to Estimate Tau and Perform Meta-Analysis with Prior
#'
#' This is a convenience wrapper that first estimates heterogeneity (tau) via REML and
#' then calls `meta_with_zero_prior_RE` to perform the meta-analysis. It includes
#' error handling for cases where tau cannot be estimated.
#'
#' @param est A numeric vector of observed effect size estimates.
#' @param se A numeric vector of observed standard errors.
#' @param n_total An integer for the total number of potential studies/samples.
#' @param c_prior A numeric scaling factor for the prior's standard deviation. Defaults to 1.
#' @param se_floor An optional minimum value for the prior's SD. If NULL (the default),
#'   it is set to the median of the observed standard errors.
#' @param method A character string for the method used by `tau_reml` to estimate tau-squared.
#' @return A data.table with meta-analysis results, or a data.table with NA values if
#'   an error occurs during calculation.
#'
combine_with_REML_prior <- function(est, se, n_total,
                                      c_prior = 1, se_floor = NULL,
                                      method = "REML") {
  
  # Handle cases with insufficient data for heterogeneity estimation.
  if (length(est) < 2) {
    # If there's only one observation, we cannot estimate between-study variance,
    # so we assume it's zero.
    tau_hat <- 0
    if (is.null(se_floor)) se_floor <- stats::median(se, na.rm = TRUE)
    
    return(meta_with_zero_prior_RE(est, se, n_total, tau_hat, c_prior = c_prior, se_floor = se_floor))
  }
  
  # Use tryCatch for the standard case with >= 2 observations, as convergence can still fail.
  tryCatch({
    
    # Estimate between-study heterogeneity (tau) from the observed data.
    tau_hat <- tau_reml(est, se, method = method)
    
    # Set a default for se_floor if not provided. Using the median observed SE
    # is a reasonable heuristic to prevent the prior from being overly strong.
    if (is.null(se_floor)) se_floor <- stats::median(se, na.rm = TRUE)
    
    # Perform the meta-analysis using the estimated tau.
    meta_with_zero_prior_RE(est, se, n_total, tau_hat, c_prior = c_prior, se_floor = se_floor)
    
  }, error = function(e) {
    # If any error occurs, return a data.table with NA values.
    # This prevents the entire analysis pipeline from crashing.
    message("REML failed to converge for a group. Details: ", e$message)
    data.table(logFC=NA_real_, logSE=NA_real_, zscore=NA_real_, pvalue=NA_real_, tau_hat=NA_real_, tau0=NA_real_)
  })
}

#' @title Calculate Cell Counts in Distance Bins from an Interface
#' @description This function takes spatial coordinates of cells and interface lines,
#'   calculates the signed distance of each cell to the nearest interface, and
#'   groups cells into discrete distance bins. It returns a matrix of cell
#'   type counts per bin for a single sample.
#' @param cells A data.table containing cell information, including 'X'/'Y' coordinates,
#'   cell type ('type_lvl3'), and a region annotation ('tessera_annotation').
#' @param interfaces An sf object containing interface geometries (e.g., LINESTRINGs).
#' @return A matrix where rows are distance bins (e.g., "(-5,0]") and columns
#'   are cell types ('type_lvl3'), with values representing cell counts.
get_bins_per_cell = function(cells, interfaces) {
    ## Get distances and closest interface type
    pts = st_as_sf(cells[, .(X, Y)], coords = c('X', 'Y'))
    geos_pts = geos::as_geos_geometry(pts$geometry)
    geos_lines = geos::as_geos_geometry(interfaces$x[1:nrow(interfaces)])
    
    nearest_interfaces_idx = geos::geos_nearest(geos_pts, geos_lines)
    
    cells$closest_interface_type = interfaces$Type_of_Interface[nearest_interfaces_idx]
    cells$dist_interface = geos::geos_distance(geos_pts, geos_lines[nearest_interfaces_idx])
    
    ## Assign sign to distances
    cells$dist_interface_signed = fifelse(
        cells$tessera_annotation == 'Stromal-enriched',
        -cells$dist_interface,
        cells$dist_interface
    )
    
    ## Assign cells to 5um bins
    dist_breaks = seq(-100, 100, by = 5)
    cells$dist_bin = cut(cells$dist_interface_signed, breaks = dist_breaks, include.lowest = TRUE)

    # --- ROBUST SUMMARIZATION --

    cells = cells %>% filter(
        (closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' & cxcl_pos_tile == 'CXCL_pos') | (closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' & cxcl_pos_tile == 'CXCL_neg')        
    )
    
    cells_in_range = cells[!is.na(dist_bin)]
    
    if (nrow(cells_in_range) == 0) {
        warning("No cells found within the -100 to 100µm distance range.")
        return(list())
    }

    all_interface_types = unique(cells$closest_interface_type)
    cells_in_range[, closest_interface_type := factor(closest_interface_type, levels = all_interface_types)]

    # counts_long = cells_in_range[, .N, by = .(closest_interface_type, dist_bin, type_lvl3)]

    # counts_wide = dcast(counts_long,
    #                     closest_interface_type + dist_bin ~ type_lvl3,
    #                     value.var = "N",
    #                     fill = 0,
    #                     drop = FALSE)

    # result_list = split(counts_wide, by = "closest_interface_type")

    # result_list = lapply(result_list, function(dt) {
    #     row_names = dt$dist_bin
    #     count_cols = setdiff(names(dt), c("closest_interface_type", "dist_bin"))
    #     mat = as.matrix(dt[, ..count_cols])
    #     rownames(mat) = row_names
    #     return(mat)
    # })

    return(cells)
}

In [ ]:
# --- Plotting Function ---

#' Plot Spatial Enrichment Results as a Heatmap
#'
#' This function takes the summarized enrichment results and generates a heatmap
#' to visualize the log-fold change of neighbor enrichment around different anchor
#' cell types. It can filter to show only significant interactions and includes
#' annotations for cell lineages.
#'
#' @param summarized_res A data.table containing the meta-analysis results, typically
#'   from `summarize_enrichment_results`. Must contain 'anchor', 'neighbor',
#'   'logFC', and 'padj' columns.
#' @return A `Heatmap` object from the ComplexHeatmap package.
#'
plot_enrichment_heatmap = function(summarized_res, binned_cells_df, col_title){
    X = summarized_res %>% 
        dcast(anchor ~ neighbor, value.var = 'logFC') %>% 
        tibble::column_to_rownames('anchor') %>% 
        as.matrix() %>% t() 
    Xsig = summarized_res %>% 
        dplyr::mutate(txt = case_when(
            is.na(pvalue) ~ '',
            padj < .05 & logFC > 0 ~ '*',
            # pvalue < 1e-3 & logFC > 0 ~ '*',
            TRUE ~ ''
        )) %>% 
        dcast(anchor ~ neighbor, value.var = 'txt') %>% 
        tibble::column_to_rownames('anchor') %>% 
        as.matrix() %>% t() 
    Xsig = Xsig[rownames(X), colnames(X)]
    
    ## Only significantly changed ones??? 
    i = as.integer(which(rowSums(Xsig == '*') > 0))
    X = X[i, , drop = FALSE]
    Xsig = Xsig[i, , drop = FALSE]
    
    i = as.integer(which(colSums(Xsig == '*') > 0))
    X = X[, i, drop = FALSE]
    Xsig = Xsig[, i, drop = FALSE]
    
    ## Maximally diagonal order
    orow = hclust(dist(X))$order
    ocol = order(max.col(t(X[orow, ])))
    X = X[orow, ocol]
    Xsig = Xsig[orow, ocol]
    
    set.seed(1)
    #fig.size(height = 4, width = 7)
    anndf = binned_cells_df %>%
        select(type_lvl1, type_lvl3) %>%
        distinct %>%
        filter(type_lvl3 %in% rownames(X)) %>%
        mutate(type_lvl3 = factor(type_lvl3, levels = rownames(X), ordered = TRUE)) %>%
        as.data.frame %>%
        rename(Lineage = type_lvl1)
    rownames(anndf) = anndf$type_lvl3
    anndf = anndf[rownames(X), ] %>% select(-type_lvl3)
    
    .X = X %>% t()
    .Xsig = Xsig %>% t()
    
    ht = Heatmap(row_names_gp = gpar(fontsize = 10),  column_names_gp = gpar(fontsize = 10),
        .X, 
        col = colorRamp2(seq(-quantile(X, .95), quantile(X, .95), length.out = 10), 
                         tableau_div_gradient_pal('Orange-Blue Diverging')(seq(1, 0, length.out = 10))), 
        rect_gp = gpar(col = "white", lwd = 2), 
        cell_fun = function(j, i, x, y, w, h, fill) {
            if (.Xsig[i, j] == '*') {
                text_col = 'white'
                grid.text("*", x = x, y = y, 
                          gp = gpar(fontsize = 16, col = text_col), 
                          hjust = 0.5, vjust = 0.75)
            }
        },    
        name = 'logFC', 
        top_annotation = columnAnnotation(df=anndf, col=list(Lineage=lineage_palette, 
                                                            nm = unique(anndf$Lineage))),
        cluster_rows = TRUE, cluster_columns = TRUE, 
        show_row_dend = FALSE, show_column_dend = FALSE, column_title = col_title
    )
    return(ht)
}

## Data Loading and Preprocessing

Here, we load the main cell data and the interface data. We perform some initial cleaning on the cell types, simplifying them into a `type_lvl3` category for the main analysis.

In [ ]:
tiles_to_omit = read.csv('../Tessera tiles/Tessera processed results/tiles_to_exclude_from_interface_analysis.csv') %>%
    filter(tiles_to_exclude_from_interface_analysis != '') %>%
    pull(agg_id)
length(tiles_to_omit)
head(tiles_to_omit)

In [ ]:
# Load cell data
cells = fread('../Tessera tiles/Tessera processed results/cells_data_for_interface_analysis.csv')
cells$type_lvl1[cells$type_lvl2 == 'Mast'] = 'Mast' 

# Simplify cell type annotations
cells <- cells %>%
    #filter(!agg_id %in% tiles_to_omit) %>%
    mutate(type_lvl2 = gsub(type_lvl2, pattern = 'Myeloid-ISGlow', replacement = 'Myeloid-ISG')) %>%
    mutate(type_lvl3 = gsub(type_lvl2, pattern = '-prolif', replacement = '')) %>% # |high|low|-PD1
    mutate(type_lvl3 = gsub(type_lvl3, pattern = 'Epi.*', replacement = 'Epi')) %>% 
    select(c('cell_id', 'cxcl_pos_tile', 'PatientID', 'SampleID', 'MMRstatus', 'X', 'Y', 'tessera_annotation', 'type_lvl3', 'type_lvl1', 'type_lvl2', 'cell_id')) 

glimpse(cells)

In [ ]:
sort(unique(cells$type_lvl2))

In [ ]:
unique(cells$PatientID[cells$MMRstatus == 'MMRd'])
unique(cells$PatientID[cells$MMRstatus == 'MMRp'])

In [ ]:
list.files(path = '../Tessera tiles/Spatial objects for tumor-stromal interfaces in all MERFISH samples/', pattern = '_tumor_stromal_interfaces.rds', full.names = TRUE)

In [ ]:
unique(cells$SampleID)

In [ ]:
# Load interface geometry files for each sample
ids = unique(cells$SampleID)
interfaces = map(ids, function(.id) {
    fname = normalizePath(list.files(path = '../Tessera tiles/Spatial objects for tumor-stromal interfaces in all MERFISH samples/', pattern = '_tumor_stromal_interfaces.rds', full.names = TRUE)[grepl(list.files(path = '../Tessera tiles/Spatial objects for tumor-stromal interfaces in all MERFISH samples/', pattern = '_tumor_stromal_interfaces.rds', full.names = TRUE), pattern = .id)])
    readRDS(fname)
})
names(interfaces) = ids

glimpse(interfaces[[1]])

In [ ]:
# Set a higher limit for global variables when using parallel processing
plan(sequential)
plan(multicore)
options(future.globals.maxSize = 1e10)

# Run get_bins for each sample in parallel
system.time({
    binned_cells = future_map(ids, function(.id) {
        get_bins_per_cell(cells[SampleID == .id], interfaces[[.id]])    
    }, .options = furrr::furrr_options(seed=TRUE))
    names(binned_cells) = ids
})

plan(sequential)

In [ ]:
binned_cells_df = rbindlist(binned_cells) %>% na.omit()
head(binned_cells_df) 

In [ ]:
# ## Remove super rare populations
# types_keep = names(which(table(binned_cells_df$type_lvl3) >= 100))
# binned_cells_df = binned_cells_df[type_lvl3 %in% types_keep]

In [ ]:
unique(binned_cells_df$type_lvl3) %>% sort()

In [ ]:
## Some global variables
BINLVLS = levels(cut(seq(-100, 100), seq(-100, 100, by = 5)))
PMMRD = unique(cells$PatientID[cells$MMRstatus == 'MMRd'])
PMMRP = unique(cells$PatientID[cells$MMRstatus == 'MMRp'])

lineage_palette = c('Epi' = '#CA49FC',
        'Strom' = '#00D2D0',
        'Myeloid' = '#FFB946',
        'Mast' = '#F4ED57',
        'Plasma' = '#61BDFC',
        'B' = '#0022FA',
        'TNKILC' = '#FF3420'
        )

reduce = purrr::reduce
map = purrr::map

In [ ]:
scales::show_col(lineage_palette)

# Around hub+ interfaces [-40, 0]: T cells as anchors

In [ ]:
dim(binned_cells_df)

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) %>%
        filter(cxcl_pos_tile == 'CXCL_pos') %>%
        filter(tessera_annotation == 'Stromal-enriched') %>%
        filter(closest_interface_type == 'CXCLpos tumor & CXCLpos stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

anchor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
neighbor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

In [ ]:
summarized_res %>% filter(neighbor == 'Plasma')

## Publication figure

In [ ]:
summarized_res <- fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
    mutate(anchor = gsub(pattern = '_', replacement = '-', x = anchor)) %>%
    mutate(neighbor = gsub(pattern = '_', replacement = '-', x = neighbor)) 

set.seed(42)
cell_counts <- binned_cells_df %>%
  filter(
    MMRstatus == "MMRd",
    dist_interface_signed >= -40,
    dist_interface_signed <= 0,
    cxcl_pos_tile == "CXCL_pos",
    closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
  ) %>%
  group_by(type_lvl3) %>%
  summarize(n = n()) %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

X <- summarized_res %>%
  dcast(anchor ~ neighbor, value.var = "logFC") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- summarized_res %>%
  dplyr::mutate(
    txt = case_when(
      is.na(pvalue) ~ "",
      padj < 0.05 & logFC > 0 ~ "*",
      # pvalue < 1e-3 & logFC > 0 ~ '*',
      TRUE ~ ""
    )
  ) %>%
  dcast(anchor ~ neighbor, value.var = "txt") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- Xsig[rownames(X), colnames(X)]

## Only significantly changed ones???
# i <- as.integer(which(rowSums(Xsig == "*") > 0))
# X <- X[i, , drop = FALSE]
# Xsig <- Xsig[i, , drop = FALSE]

# i <- as.integer(which(colSums(Xsig == '*') > 0))
# X <- X[, i, drop = FALSE]
# Xsig <- Xsig[, i, drop = FALSE]
set.seed(1)

anndf <- binned_cells_df %>%
  select(type_lvl1, type_lvl3) %>%
  distinct() %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3)) %>%
  # filter(type_lvl3 %in% rownames(X)) %>%
  # mutate(type_lvl3 = factor(type_lvl3, levels = rownames(X), ordered = TRUE)) %>%
  as.data.frame() %>%
  rename(Anchors = type_lvl1)
rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
rownames(anndf) = str_wrap(rownames(anndf),  width = 15, whitespace_only = FALSE)

anndf_neighbors <- anndf %>% 
  rename(Neighbors = Anchors)

.X <- t(X)
.Xsig <- t(Xsig)

ordered_row_counts <- cell_counts_vec[rownames(.X)]
formatted_row_counts <- formatC(ordered_row_counts, big.mark = ",", format = "d")

ordered_col_counts <- cell_counts_vec[colnames(.X)]
# Format numbers with commas
formatted_col_counts <- formatC(ordered_col_counts, big.mark = ",", format = "d")

colnames(.X) = str_wrap(colnames(.X), width = 15, whitespace_only = FALSE)
rownames(.X) = str_wrap(rownames(.X), width = 15, whitespace_only = FALSE)
colnames(.Xsig) = str_wrap(colnames(.Xsig), width = 15, whitespace_only = FALSE)
rownames(.Xsig) = str_wrap(rownames(.Xsig), width = 15, whitespace_only = FALSE)
names(cell_counts_vec) = str_wrap(names(cell_counts_vec), width = 15, whitespace_only = FALSE)

# --- Heatmap Generation ---

ht <- Heatmap(
  .X,
  # Parameters for the main heatmap legend
  name = "logFC",
  heatmap_legend_param = list(
    direction = "horizontal" # Set main legend to horizontal
  ),
  
  # Other parameters...
  row_km = 2,
  bottom_annotation = HeatmapAnnotation(which = 'column',
    annotation_name_side = "left",
    annotation_name_rot = 0,
    Neighbors = anndf_neighbors[colnames(.X), ],
    col = list(Neighbors = lineage_palette), 
    annotation_legend_param = list(
        Neighbors = list(direction = "horizontal")
    )
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(
      formatted_col_counts,
      rot = 90,
     # gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "left"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #height = max_text_width(formatted_col_counts) + unit(4, "mm")
  ),
  left_annotation = rowAnnotation(
    `N Cells` = anno_text(
      formatted_row_counts,
      just = "right",
      location = 1,
      #gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "bottom"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #width = max_text_width(formatted_row_counts) + unit(4, "mm")
  ),
  right_annotation = rowAnnotation(
    annotation_name_side = "top",
    annotation_name_rot = 0,
    Anchors = anndf[rownames(.X), ],
    col = list(Anchors = lineage_palette),
    # Parameters for this specific annotation's legend
    annotation_legend_param = list(
      Anchors = list(direction = "horizontal") 
    )
  ),
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  #row_names_gp = gpar(fontsize = 10),
  #column_names_gp = gpar(fontsize = 10),
  col = colorRamp2(
    seq(-quantile(X, 0.95), quantile(X, 0.95), length.out = 10),
    tableau_div_gradient_pal("Orange-Blue Diverging")(seq(1, 0, length.out = 10))
  ),
  rect_gp = gpar(col = "white", lwd = 2),
  cell_fun = function(j, i, x, y, w, h, fill) {
    if (.Xsig[i, j] == "*") {
      text_col <- "white"
      grid.text(
        "*", x = x, y = y,
        gp = gpar(col = text_col), #fontsize = 10, 
        hjust = 0.5, vjust = 0.75
      )
    }
  },

  show_row_dend = FALSE,
  show_column_dend = FALSE
)

# --- Draw the Heatmap 
set.seed(2)
fig.size(height = 7, width = 10, res = 400)
draw(ht, heatmap_legend_side = 'bottom', annotation_legend_side = 'bottom', merge_legend = TRUE)

# --- Draw the Heatmap 
set.seed(2)
fig.size(height = 7, width = 10, res = 400)
pdf('figure6A_EPI_COLLAPSED.pdf', height = 7, width = 10)
draw(ht, heatmap_legend_side = 'bottom', annotation_legend_side = 'bottom', merge_legend = TRUE)
dev.off()

In [ ]:
summarized_res <- fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
    mutate(anchor = gsub(pattern = '_', replacement = '-', x = anchor)) %>%
    mutate(neighbor = gsub(pattern = '_', replacement = '-', x = neighbor)) 

set.seed(42)
cell_counts <- binned_cells_df %>%
  filter(
    MMRstatus == "MMRd",
    dist_interface_signed >= -40,
    dist_interface_signed <= 0,
    cxcl_pos_tile == "CXCL_pos",
    closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
  ) %>%
  group_by(type_lvl3) %>%
  summarize(n = n()) %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

X <- summarized_res %>%
  dcast(anchor ~ neighbor, value.var = "logFC") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- summarized_res %>%
  dplyr::mutate(
    txt = case_when(
      is.na(pvalue) ~ "",
      padj < 0.05 & logFC > 0 ~ "*",
      # pvalue < 1e-3 & logFC > 0 ~ '*',
      TRUE ~ ""
    )
  ) %>%
  dcast(anchor ~ neighbor, value.var = "txt") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- Xsig[rownames(X), colnames(X)]

## Only significantly changed ones???
# i <- as.integer(which(rowSums(Xsig == "*") > 0))
# X <- X[i, , drop = FALSE]
# Xsig <- Xsig[i, , drop = FALSE]

# i <- as.integer(which(colSums(Xsig == '*') > 0))
# X <- X[, i, drop = FALSE]
# Xsig <- Xsig[, i, drop = FALSE]
set.seed(1)

anndf <- binned_cells_df %>%
  select(type_lvl1, type_lvl3) %>%
  distinct() %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3)) %>%
  # filter(type_lvl3 %in% rownames(X)) %>%
  # mutate(type_lvl3 = factor(type_lvl3, levels = rownames(X), ordered = TRUE)) %>%
  as.data.frame() %>%
  rename(Anchors = type_lvl1)
rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
rownames(anndf) = str_wrap(rownames(anndf),  width = 15, whitespace_only = FALSE)

anndf_neighbors <- anndf %>% 
  rename(Neighbors = Anchors)

.X <- t(X)
.Xsig <- t(Xsig)

ordered_row_counts <- cell_counts_vec[rownames(.X)]
formatted_row_counts <- formatC(ordered_row_counts, big.mark = ",", format = "d")

ordered_col_counts <- cell_counts_vec[colnames(.X)]
# Format numbers with commas
formatted_col_counts <- formatC(ordered_col_counts, big.mark = ",", format = "d")

colnames(.X) = str_wrap(colnames(.X), width = 15, whitespace_only = FALSE)
rownames(.X) = str_wrap(rownames(.X), width = 15, whitespace_only = FALSE)
colnames(.Xsig) = str_wrap(colnames(.Xsig), width = 15, whitespace_only = FALSE)
rownames(.Xsig) = str_wrap(rownames(.Xsig), width = 15, whitespace_only = FALSE)
names(cell_counts_vec) = str_wrap(names(cell_counts_vec), width = 15, whitespace_only = FALSE)

# --- Heatmap Generation ---

ht <- Heatmap(
  .X,
  # Parameters for the main heatmap legend
  name = "logFC",
  heatmap_legend_param = list(
    direction = "horizontal" # Set main legend to horizontal
  ),
  
  # Other parameters...
  row_km = 2,
  bottom_annotation = HeatmapAnnotation(which = 'column',
    annotation_name_side = "left",
    annotation_name_rot = 0,
    Neighbors = anndf_neighbors[colnames(.X), ],
    col = list(Neighbors = lineage_palette), 
    annotation_legend_param = list(
        Neighbors = list(direction = "horizontal")
    )
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(
      formatted_col_counts,
      rot = 90,
     # gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "left"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #height = max_text_width(formatted_col_counts) + unit(4, "mm")
  ),
  left_annotation = rowAnnotation(
    `N Cells` = anno_text(
      formatted_row_counts,
      just = "right",
      location = 1,
      #gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "bottom"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #width = max_text_width(formatted_row_counts) + unit(4, "mm")
  ),
  right_annotation = rowAnnotation(
    annotation_name_side = "top",
    annotation_name_rot = 0,
    Anchors = anndf[rownames(.X), ],
    col = list(Anchors = lineage_palette),
    # Parameters for this specific annotation's legend
    annotation_legend_param = list(
      Anchors = list(direction = "horizontal") 
    )
  ),
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  #row_names_gp = gpar(fontsize = 10),
  #column_names_gp = gpar(fontsize = 10),
  col = colorRamp2(
    seq(-quantile(X, 0.95), quantile(X, 0.95), length.out = 10),
    tableau_div_gradient_pal("Orange-Blue Diverging")(seq(1, 0, length.out = 10))
  ),
  rect_gp = gpar(col = "white", lwd = 2),
  cell_fun = function(j, i, x, y, w, h, fill) {
    if (.Xsig[i, j] == "*") {
      text_col <- "white"
      grid.text(
        "*", x = x, y = y,
        gp = gpar(col = text_col), #fontsize = 10, 
        hjust = 0.5, vjust = 0.75
      )
    }
  },

  show_row_dend = FALSE,
  show_column_dend = FALSE
)

# --- Draw the Heatmap 
set.seed(2)
fig.size(height = 7, width = 20, res = 400)
draw(ht, heatmap_legend_side = 'bottom', annotation_legend_side = 'bottom', merge_legend = TRUE)


ht_anchor = ht

# Around hub+ interfaces [-40, 0]: T cells as neighbors

In [ ]:
dim(binned_cells_df)

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) %>%
        filter(cxcl_pos_tile == 'CXCL_pos') %>%
        filter(tessera_annotation == 'Stromal-enriched') %>%
        filter(closest_interface_type == 'CXCLpos tumor & CXCLpos stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

neighbor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
anchor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

In [ ]:
summarized_res %>% filter(neighbor == 'Plasma')

## Publication figure

In [ ]:
summarized_res <- fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>%
    mutate(anchor = gsub(pattern = '_', replacement = '-', x = anchor)) %>%
    mutate(neighbor = gsub(pattern = '_', replacement = '-', x = neighbor)) 

set.seed(42)
cell_counts <- binned_cells_df %>%
  filter(
    MMRstatus == "MMRd",
    dist_interface_signed >= -40,
    dist_interface_signed <= 0,
    cxcl_pos_tile == "CXCL_pos",
    closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
  ) %>%
  group_by(type_lvl3) %>%
  summarize(n = n()) %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

X <- summarized_res %>%
  dcast(anchor ~ neighbor, value.var = "logFC") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- summarized_res %>%
  dplyr::mutate(
    txt = case_when(
      is.na(pvalue) ~ "",
      padj < 0.05 & logFC > 0 ~ "*",
      # pvalue < 1e-3 & logFC > 0 ~ '*',
      TRUE ~ ""
    )
  ) %>%
  dcast(anchor ~ neighbor, value.var = "txt") %>%
  tibble::column_to_rownames("anchor") %>%
  as.matrix() %>%
  t()

Xsig <- Xsig[rownames(X), colnames(X)]

## Only significantly changed ones???
i <- as.integer(which(rowSums(Xsig == "*") > 0))
X <- X[i, , drop = FALSE]
Xsig <- Xsig[i, , drop = FALSE]

i <- as.integer(which(colSums(Xsig == '*') > 0))
X <- X[, i, drop = FALSE]
Xsig <- Xsig[, i, drop = FALSE]
set.seed(1)

anndf <- binned_cells_df %>%
  select(type_lvl1, type_lvl3) %>%
  distinct() %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3)) %>%
  # filter(type_lvl3 %in% rownames(X)) %>%
  # mutate(type_lvl3 = factor(type_lvl3, levels = rownames(X), ordered = TRUE)) %>%
  as.data.frame() %>%
  rename(Neighbors = type_lvl1) # Anchors
rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
rownames(anndf) = str_wrap(rownames(anndf),  width = 15, whitespace_only = FALSE)

anndf_neighbors <- anndf %>% 
  rename(Anchors = Neighbors)
  #rename(Neighbors = Anchors)

.X <- X #t(X)
.Xsig <-Xsig # t(Xsig)

ordered_row_counts <- cell_counts_vec[rownames(.X)]
formatted_row_counts <- formatC(ordered_row_counts, big.mark = ",", format = "d")

ordered_col_counts <- cell_counts_vec[colnames(.X)]
# Format numbers with commas
formatted_col_counts <- formatC(ordered_col_counts, big.mark = ",", format = "d")

colnames(.X) = str_wrap(colnames(.X), width = 15, whitespace_only = FALSE)
rownames(.X) = str_wrap(rownames(.X), width = 15, whitespace_only = FALSE)
colnames(.Xsig) = str_wrap(colnames(.Xsig), width = 15, whitespace_only = FALSE)
rownames(.Xsig) = str_wrap(rownames(.Xsig), width = 15, whitespace_only = FALSE)
names(cell_counts_vec) = str_wrap(names(cell_counts_vec), width = 15, whitespace_only = FALSE)

# --- Heatmap Generation ---

ht <- Heatmap(
  .X,
  # Parameters for the main heatmap legend
  name = "logFC",
  heatmap_legend_param = list(
    direction = "horizontal" # Set main legend to horizontal
  ),
  
  # Other parameters...
  row_km = 2,
  bottom_annotation = HeatmapAnnotation(which = 'column',
    annotation_name_side = "left",
    annotation_name_rot = 0,
    Anchors = anndf_neighbors[colnames(.X), ],
    col = list(Anchors = lineage_palette), 
    annotation_legend_param = list(
        Anchors = list(direction = "horizontal")
    )
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(
      formatted_col_counts,
      rot = 90,
     # gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "left"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #height = max_text_width(formatted_col_counts) + unit(4, "mm")
  ),
  left_annotation = rowAnnotation(
    `N Cells` = anno_text(
      formatted_row_counts,
      just = "right",
      location = 1,
      #gp = gpar(fontsize = 8)
    ),
    annotation_name_side = "bottom"#,
    #annotation_name_gp = gpar(fontsize = 8),
    #width = max_text_width(formatted_row_counts) + unit(4, "mm")
  ),
  right_annotation = rowAnnotation(
    annotation_name_side = "top",
    annotation_name_rot = 0,
    Neighbors = anndf[rownames(.X), ],
    col = list(Neighbors = lineage_palette),
    # Parameters for this specific annotation's legend
    annotation_legend_param = list(
      Neighbors = list(direction = "horizontal") 
    )
  ),
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  #row_names_gp = gpar(fontsize = 10),
  #column_names_gp = gpar(fontsize = 10),
  col = colorRamp2(
    seq(-quantile(X, 0.95), quantile(X, 0.95), length.out = 10),
    tableau_div_gradient_pal("Orange-Blue Diverging")(seq(1, 0, length.out = 10))
  ),
  rect_gp = gpar(col = "white", lwd = 2),
  cell_fun = function(j, i, x, y, w, h, fill) {
    if (.Xsig[i, j] == "*") {
      text_col <- "white"
      grid.text(
        "*", x = x, y = y,
        gp = gpar(col = text_col), #fontsize = 10, 
        hjust = 0.5, vjust = 0.75
      )
    }
  },

  show_row_dend = FALSE,
  show_column_dend = FALSE
)

# --- Draw the Heatmap 
set.seed(2)
fig.size(height = 7, width = 20, res = 400)
draw(ht, heatmap_legend_side = 'bottom', annotation_legend_side = 'bottom', merge_legend = TRUE)

ht_neighbor = ht

# Combined heatmap

In [ ]:
anchor_list = c(fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor),
                fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor)) %>% 
            unique

neighbor_list = c(fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor),
                fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor)) %>% 
            unique

anchor_list %>% length
neighbor_list %>% length

In [ ]:
anndf <- binned_cells_df %>%
  select(type_lvl1, type_lvl3) %>%
  distinct() %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3)) %>%
  as.data.frame() %>%
  rename(Anchors = type_lvl1) # Anchors
rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
rownames(anndf) = str_wrap(rownames(anndf),  width = 15, whitespace_only = FALSE)

anndf_neighbors <- anndf %>% 
  rename(Neighbors = Anchors)

anndf

In [ ]:
## -----------------------------------------------------------------------------
## Script: Aligned and Concatenated Interaction Heatmaps
##
## This script generates two heatmaps of cell-cell interaction logFC values,
## one for "T cells as anchor" and one for "T cells as neighbor". It ensures
## that both heatmaps are perfectly aligned by row and column, removes any
## columns that are empty in both datasets, and concatenates them vertically
## for a final comparative visualization.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

# Load necessary packages for data manipulation and visualization
library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer) # Added for the color palette


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)


# Create a vector of cell counts to use in the 'N Cells' annotation
cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRd",
   dist_interface_signed >= -40,
   dist_interface_signed <= 0,
   cxcl_pos_tile == "CXCL_pos",
   closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 # Standardize cell type names by replacing underscores and shortening names
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

# Convert to a named vector for easy lookup
cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

# Create annotation data frames for the heatmap sidebars (cell lineages)
anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

# Define the canonical order and splits for rows (T-cell subtypes)
row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
# Define the canonical order and splits for columns (all other cell types)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
# Store the full, desired order of rows and columns. This will be filtered later.
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec) # Store the initial order before filtering


# --- 3. Process Raw Data Files ---

## 3.1 Load and clean "T as Anchor" data
summarized_res_anchor <- fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data from long to wide format for the heatmap matrix
X_anchor_raw <- dcast(
 summarized_res_anchor, anchor ~ neighbor,
 value.var = "logFC"
) %>%
 column_to_rownames("anchor") %>%
 as.matrix()

# Create a corresponding matrix for significance stars
Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>%
 as.matrix()

## 3.2 Load and clean "T as Neighbor" data
summarized_res_neighbor <- fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data (NOTE: formula is swapped to keep T cells as rows)
X_neighbor_raw <- dcast(
 summarized_res_neighbor, neighbor ~ anchor,
 value.var = "logFC"
) %>%
 column_to_rownames("neighbor") %>%
 as.matrix()

# Create the significance matrix for the neighbor data
Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>%
 as.matrix()


# --- 4. Filter and Align Matrices ---

## 4.1 Find and keep columns that have significant interactions in EITHER dataset
# Find column names with at least one star in the 'anchor' data
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
# Find column names with at least one star in the 'neighbor' data
sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
# Combine the two lists to get all columns that are significant in at least one heatmap
all_significant_cols <- union(sig_cols_anchor, sig_cols_neighbor)
# Filter your desired column order to only keep columns from the significant list
final_col_order <- intersect(initial_col_order, all_significant_cols)


## 4.2 Create perfectly aligned matrices using the filtered column order
# Create empty matrices with the final desired shape and order
X_anchor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_anchor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
X_neighbor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_neighbor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)

# Find common cells between raw and aligned matrices and populate the data
common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)

X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]

# FIX: Replace NA values with empty strings to prevent errors in the plotting function
Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""


# --- 5. Prepare Final Plotting Objects ---

# Apply string wrapping to all labels for consistent, clean plotting
rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(Xsig_anchor_aligned) <- dimnames(X_anchor_aligned)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_neighbor_aligned) <- dimnames(X_anchor_aligned)

# Update annotation data frames with wrapped labels
rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
anndf$Anchors <- str_wrap(anndf$Anchors, width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)
anndf_neighbors$Neighbors <- str_wrap(anndf_neighbors$Neighbors, width = 20)

# Update and filter split vectors with wrapped labels
names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order] # Filter to match new columns
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

# Format cell count numbers with commas for annotations
# Filter the original count vector to match the final row/column order
formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")


# --- 6. Generate Heatmaps ---

# Define common legend parameters for a consistent look
legend_params <- list(
 title_gp = gpar(fontsize = 7),
 labels_gp = gpar(fontsize = 7),
 direction = "vertical"
)

# Define a shared color function so the color scale is identical for both heatmaps
value_range <- quantile(c(X_anchor_aligned, X_neighbor_aligned), c(0.05, 0.95), na.rm = TRUE)
col_fun <- colorRamp2(
 seq(value_range[1], value_range[2], length.out = 10),
 rev(RColorBrewer::brewer.pal(10, "PuOr"))
)

## 6.1 Heatmap 1: T as Anchor
ht_anchor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_anchor_aligned,
 name = "Colocalization\nlogFC",
 col = col_fun,

 # Clustering and ordering
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_anchor_aligned)],
 column_split = column_slices_vec[colnames(X_anchor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),
 heatmap_legend_param = legend_params,

 # Annotations
 top_annotation = HeatmapAnnotation(
   `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
   annotation_name_side = "left",
   annotation_name_gp = gpar(fontsize = 8)
 ),
 right_annotation = rowAnnotation(
   show_legend = FALSE,
   Anchors = anndf[rownames(X_anchor_aligned), "Anchors", drop = TRUE],
   col = list(Anchors = lineage_palette),
   annotation_legend_param = list(Anchors = legend_params),
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),

 # Custom function to draw significance stars in cells
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_anchor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

## 6.2 Heatmap 2: T as Neighbor
ht_neighbor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_neighbor_aligned,
 name = "logFC_neighbor",
 col = col_fun,

 # Hide redundant elements for a cleaner concatenated plot
 show_heatmap_legend = FALSE,
 show_row_names = TRUE,
 show_column_names = TRUE,

 # Clustering and ordering (must be identical to the first heatmap)
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_neighbor_aligned)],
 column_split = column_slices_vec[colnames(X_neighbor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),

 # Annotations (with legends hidden)
 right_annotation = rowAnnotation(
   Neighbors = anndf[rownames(X_neighbor_aligned), "Anchors", drop = TRUE],
   col = list(Neighbors = lineage_palette),
   show_legend = FALSE,
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 bottom_annotation = HeatmapAnnotation(
   show_legend = TRUE,
   Lineage = anndf_neighbors[colnames(X_anchor_aligned), "Neighbors", drop = TRUE],
   annotation_name_gp = gpar(fontsize = 0),
   col = list(Lineage = lineage_palette),
   annotation_legend_param = list(Lineage = legend_params),
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 # Custom cell function for stars
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_neighbor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

# --- 7. Concatenate and Draw Final Plot ---

# Combine the two heatmaps vertically using the %v% operator
set.seed(42)
ht_list <- ht_anchor %v% ht_neighbor

# Draw the final, combined plot

fig.size(height = 5, width = 7)
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)

#pdf('figure6A.pdf', width = 7, height = 5,  useKerning = TRUE)
# Draw the final, combined plot
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)
#dev.off()

# PUBLICATION HEATMAP: Make a heatmap with T cells as neighbors, with red stars showing the pairs significant in both directions

In [ ]:
## -----------------------------------------------------------------------------
## Script: Integrated Interaction Heatmap
##
## This script generates a single heatmap of cell-cell interaction logFC values,
## displaying data for "T cells as neighbor". It uses a multi-color system of
## stars to indicate whether an interaction is significant in the anchor data,
## the neighbor data, or both.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)

cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRd",
   dist_interface_signed >= -40,
   dist_interface_signed <= 0,
   cxcl_pos_tile == "CXCL_pos",
   closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec)


# --- 3. Process Raw Data Files ---
summarized_res_anchor <- fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_anchor_raw <- dcast(summarized_res_anchor, anchor ~ neighbor, value.var = "logFC") %>%
 column_to_rownames("anchor") %>% as.matrix()

Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>% as.matrix()

summarized_res_neighbor <- fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_neighbor_raw <- dcast(summarized_res_neighbor, neighbor ~ anchor, value.var = "logFC") %>%
 column_to_rownames("neighbor") %>% as.matrix()

Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>% as.matrix()


# --- 4. Filter and Align Matrices ---
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
all_significant_cols <- union(sig_cols_anchor, sig_cols_neighbor)
final_col_order <- intersect(initial_col_order, all_significant_cols)

X_anchor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_anchor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
X_neighbor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_neighbor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))

common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)
X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_anchor_raw[common_rows_neighbor, common_cols_neighbor] #X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]


# --- 5. Create Combined Significance Matrix & Prepare Objects ---

Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""

Xsig_combined <- matrix("None",
                        nrow = nrow(Xsig_anchor_aligned),
                        ncol = ncol(Xsig_anchor_aligned),
                        dimnames = dimnames(Xsig_anchor_aligned))

Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned != "*"] <- "Anchor Only"
Xsig_combined[Xsig_anchor_aligned != "*" & Xsig_neighbor_aligned == "*"] <- "Neighbor Only"
Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned == "*"] <- "Both"

rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_combined) <- dimnames(X_anchor_aligned)

rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)

names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order]
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")


# --- 6. Generate the Integrated Heatmap ---

star_colors <- c("Both" = "red", "Anchor Only" = "blue", "Neighbor Only" = "black")

# --- UPDATED COLOR LOGIC START ---
# 1. Combine data to find the global range
all_values <- c(X_anchor_aligned, X_neighbor_aligned)

# 2. Find the maximum absolute value of the 5th/95th percentiles.
#    This creates a symmetric limit (e.g., if quantile range is -2 to 5, limit becomes 5).
#    This ensures -limit and +limit are equidistant from 0.
limit <- max(abs(quantile(all_values, c(0.05, 0.95), na.rm = TRUE)))

# 3. Define the palette.
#    We use 11 colors (max for PuOr) to get the deepest purple and orange.
#    We keep your 'rev' logic (Purple=Low, Orange=High).
puor_colors <- rev(RColorBrewer::brewer.pal(8, "PuOr"))

# 4. Create the mapping with 3 fixed anchors: Min -> 0 (White) -> Max
col_fun <- colorRamp2(
  c(-limit, 0, limit),
  c(puor_colors[1], "white", puor_colors[8])
)
# --- UPDATED COLOR LOGIC END ---

ht_combined <- Heatmap(
  X_neighbor_aligned,
  name = "Colocalization logFC -\nT cells as anchor",
  col = col_fun,
  cluster_rows = FALSE,
  cluster_columns = TRUE,
  cluster_column_slices = FALSE,
  show_column_dend = FALSE,
  row_split = row_slices_vec[rownames(X_neighbor_aligned)],
  column_split = column_slices_vec[colnames(X_neighbor_aligned)],
  rect_gp = gpar(col = "white", lwd = 1),
  row_names_gp = gpar(fontsize = 7),
  column_names_gp = gpar(fontsize = 7),
  row_title_gp = gpar(fontsize = 0),
  column_title_gp = gpar(fontsize = 0),
  heatmap_legend_param = list(
      title_gp = gpar(fontsize=7, fontface = "bold"), 
      labels_gp = gpar(fontsize=7),
      direction = "horizontal" # MODIFIED
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
    annotation_name_side = "left",
    annotation_name_gp = gpar(fontsize = 7)
  ),
  bottom_annotation = HeatmapAnnotation(
    show_legend = TRUE,
    Lineage = anndf_neighbors[colnames(X_neighbor_aligned), "Neighbors", drop = TRUE],
    annotation_name_gp = gpar(fontsize = 0),
    col = list(Lineage = lineage_palette),
    annotation_legend_param = list(
        Lineage = list(
            title_gp = gpar(fontsize=7, fontface = "bold"), 
            labels_gp = gpar(fontsize=7),
            direction = "horizontal",
            ncol = 2 # MODIFIED: Arrange legend into 2 columns
        )
    ),
    simple_anno_size = unit(2, "mm")
  ),
  cell_fun = function(j, i, x, y, w, h, fill) {
    status <- Xsig_combined[i, j]
    if (status != "None") {
      grid.text("*", x, y, gp = gpar(col = star_colors[status], fontsize = 13))
    }
  }
)
# --- 7. Create a Custom Legend for the Stars ---

star_legend <- Legend(
  labels = c("Significant in Both", "Significant in T\nas Anchor Only", "Significant in T\nas Neighbor Only"),
  title = "Colocalization",
  legend_gp = gpar(col = c("red", "blue", "black"), fontsize = 13),
  pch = "*",
  type = "points",
  labels_gp = gpar(fontsize = 7),
  title_gp = gpar(fontsize = 7, fontface = "bold"),
  direction = "horizontal" # MODIFIED
)


# --- 8. Draw the Final Plot ---
fig.size(height = 3.15, width = 6.5, res = 400)
draw(
  ht_combined,
  heatmap_legend_side = "right",
  annotation_legend_side = "right",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)

pdf('figure6A_merged_colors.pdf', width = 6.5, height = 3.15,  useKerning = TRUE)
draw(
  ht_combined,
  heatmap_legend_side = "right", #"right",
  annotation_legend_side = "right", #"right",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)
dev.off()

# Show only red and white stars

In [ ]:
## -----------------------------------------------------------------------------
## Script: Integrated Interaction Heatmap
##
## This script generates a single heatmap of cell-cell interaction logFC values,
## displaying data for "T cells as neighbor". It uses a multi-color system of
## stars to indicate whether an interaction is significant in the anchor data,
## the neighbor data, or both.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)

cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRd",
   dist_interface_signed >= -40,
   dist_interface_signed <= 0,
   cxcl_pos_tile == "CXCL_pos",
   closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec)


# --- 3. Process Raw Data Files ---
summarized_res_anchor <- fread("0_to_neg40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_anchor_raw <- dcast(summarized_res_anchor, anchor ~ neighbor, value.var = "logFC") %>%
 column_to_rownames("anchor") %>% as.matrix()

Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>% as.matrix()

summarized_res_neighbor <- fread("0_to_neg40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_neighbor_raw <- dcast(summarized_res_neighbor, neighbor ~ anchor, value.var = "logFC") %>%
 column_to_rownames("neighbor") %>% as.matrix()

Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>% as.matrix()


# --- 4. Filter and Align Matrices ---
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
#sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
all_significant_cols <- sig_cols_anchor #union(sig_cols_anchor, sig_cols_neighbor)
final_col_order <- intersect(initial_col_order, all_significant_cols)

X_anchor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_anchor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
X_neighbor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_neighbor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))

common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)
X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_anchor_raw[common_rows_neighbor, common_cols_neighbor] #X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]


# --- 5. Create Combined Significance Matrix & Prepare Objects ---

Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""

Xsig_combined <- matrix("None",
                        nrow = nrow(Xsig_anchor_aligned),
                        ncol = ncol(Xsig_anchor_aligned),
                        dimnames = dimnames(Xsig_anchor_aligned))

Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned != "*"] <- "Anchor Only"
#Xsig_combined[Xsig_anchor_aligned != "*" & Xsig_neighbor_aligned == "*"] <- "Neighbor Only"
Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned == "*"] <- "Both"

rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_combined) <- dimnames(X_anchor_aligned)

rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)

names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order]
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")


# --- 6. Generate the Integrated Heatmap ---

star_colors <- c("Both" = "red", 
                 "Anchor Only" = "blue"#, 
                # "Neighbor Only" = "black"
                )

# --- UPDATED COLOR LOGIC START ---
# 1. Combine data to find the global range
all_values <- c(X_anchor_aligned, X_neighbor_aligned)

# 2. Find the maximum absolute value of the 5th/95th percentiles.
#    This creates a symmetric limit (e.g., if quantile range is -2 to 5, limit becomes 5).
#    This ensures -limit and +limit are equidistant from 0.
limit <- max(abs(quantile(all_values, c(0.05, 0.95), na.rm = TRUE)))

# 3. Define the palette.
#    We use 11 colors (max for PuOr) to get the deepest purple and orange.
#    We keep your 'rev' logic (Purple=Low, Orange=High).
puor_colors <- rev(RColorBrewer::brewer.pal(8, "PuOr"))

# 4. Create the mapping with 3 fixed anchors: Min -> 0 (White) -> Max
col_fun <- colorRamp2(
  c(-limit, 0, limit),
  c(puor_colors[1], "white", puor_colors[8])
)
# --- UPDATED COLOR LOGIC END ---

ht_combined <- Heatmap(
  X_neighbor_aligned,
  name = "Colocalization logFC -\nT cells as anchor",
  col = col_fun,
  cluster_rows = FALSE,
  cluster_columns = TRUE,
  cluster_column_slices = FALSE,
  show_column_dend = FALSE,
  row_split = row_slices_vec[rownames(X_neighbor_aligned)],
  column_split = column_slices_vec[colnames(X_neighbor_aligned)],
  rect_gp = gpar(col = "white", lwd = 1),
  row_names_gp = gpar(fontsize = 7),
  column_names_gp = gpar(fontsize = 7),
  row_title_gp = gpar(fontsize = 0),
  column_title_gp = gpar(fontsize = 0),
  heatmap_legend_param = list(
      title_gp = gpar(fontsize=7, fontface = "bold"), 
      labels_gp = gpar(fontsize=7),
      direction = "horizontal" # MODIFIED
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
    annotation_name_side = "left",
    annotation_name_gp = gpar(fontsize = 7)
  ),
  bottom_annotation = HeatmapAnnotation(
    show_legend = TRUE,
    Lineage = anndf_neighbors[colnames(X_neighbor_aligned), "Neighbors", drop = TRUE],
    annotation_name_gp = gpar(fontsize = 0),
    col = list(Lineage = lineage_palette),
    annotation_legend_param = list(
        Lineage = list(
            title_gp = gpar(fontsize=7, fontface = "bold"), 
            labels_gp = gpar(fontsize=7),
            direction = "horizontal",
            ncol = 2 # MODIFIED: Arrange legend into 2 columns
        )
    ),
    simple_anno_size = unit(2, "mm")
  ),
  cell_fun = function(j, i, x, y, w, h, fill) {
    status <- Xsig_combined[i, j]
    if (status != "None") {
      grid.text("*", x, y, gp = gpar(col = star_colors[status], fontsize = 13))
    }
  }
)
# --- 7. Create a Custom Legend for the Stars ---

star_legend <- Legend(
  labels = c("Significant in Both", 
             "Significant in T\nas Anchor Only"#, 
             #"Significant in T\nas Neighbor Only"
            ),
  title = "Significance",
  legend_gp = gpar(col = c("red", "blue"), 
                           #"black"),
                   fontsize = 13),
  pch = "*",
  type = "points",
  labels_gp = gpar(fontsize = 7),
  title_gp = gpar(fontsize = 7, fontface = "bold"),
  direction = "horizontal" # MODIFIED
)


# --- 8. Draw the Final Plot ---
fig.size(height = 3.15, width = 6.5, res = 400)
draw(
  ht_combined,
  heatmap_legend_side = "right",
  annotation_legend_side = "right",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)

pdf('figure6A_only_T_as_anchor.pdf', width = 6.5, height = 3.15,  useKerning = TRUE)
draw(
  ht_combined,
  heatmap_legend_side = "right", #"right",
  annotation_legend_side = "right", #"right",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)
dev.off()

# Supplementary figure showing colocalization around the hub+ interface on the EPITHELIAL side

## Around hub+ interfaces [0, 40]: T cells as anchors

In [ ]:
binned_cells_df %>% glimpse()

In [ ]:
binned_cells_df$dist_interface_signed %>% range
binned_cells_df$tessera_annotation %>% unique

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > 0) %>%
        filter(dist_interface_signed < 40) %>%
        filter(cxcl_pos_tile == 'CXCL_pos') %>%
        filter(tessera_annotation == 'Epithelial-enriched') %>%
        filter(closest_interface_type == 'CXCLpos tumor & CXCLpos stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

anchor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
neighbor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_40_hub_T_as_anchor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Around hub+ interfaces [0, 40]: T cells as neighbors

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > 0) %>%
        filter(dist_interface_signed < 40) %>%
        filter(cxcl_pos_tile == 'CXCL_pos') %>%
        filter(tessera_annotation == 'Epithelial-enriched') %>%
        filter(closest_interface_type == 'CXCLpos tumor & CXCLpos stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

neighbor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
anchor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_40_hub_T_as_neighbor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Combined heatmap

In [ ]:
anchor_list = c(fread("0_to_40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor),
                fread("0_to_40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor)) %>% 
            unique

neighbor_list = c(fread("0_to_40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor),
                fread("0_to_40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor)) %>% 
            unique

anchor_list %>% length
neighbor_list %>% length

In [ ]:
anndf <- binned_cells_df %>%
  select(type_lvl1, type_lvl3) %>%
  distinct() %>%
  mutate(type_lvl3 = gsub(pattern = '_', replacement = '-', x = type_lvl3)) %>%
  as.data.frame() %>%
  rename(Anchors = type_lvl1) # Anchors
rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
rownames(anndf) = str_wrap(rownames(anndf),  width = 15, whitespace_only = FALSE)

anndf_neighbors <- anndf %>% 
  rename(Neighbors = Anchors)

anndf

# PUBLICATION HEATMAP: Make a heatmap with T cells as neighbors, with red stars showing the pairs significant in both directions

In [ ]:
## -----------------------------------------------------------------------------
## Script: Integrated Interaction Heatmap
##
## This script generates a single heatmap of cell-cell interaction logFC values,
## displaying data for "T cells as neighbor". It uses a multi-color system of
## stars to indicate whether an interaction is significant in the anchor data,
## the neighbor data, or both.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)

cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRd",
   dist_interface_signed > 0,
   dist_interface_signed < 40,
   cxcl_pos_tile == "CXCL_pos",
   closest_interface_type == "CXCLpos tumor & CXCLpos stroma"
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec)


# --- 3. Process Raw Data Files ---
summarized_res_anchor <- fread("0_to_40_hub_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_anchor_raw <- dcast(summarized_res_anchor, anchor ~ neighbor, value.var = "logFC") %>%
 column_to_rownames("anchor") %>% as.matrix()

Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>% as.matrix()

summarized_res_neighbor <- fread("0_to_40_hub_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

X_neighbor_raw <- dcast(summarized_res_neighbor, neighbor ~ anchor, value.var = "logFC") %>%
 column_to_rownames("neighbor") %>% as.matrix()

Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>% as.matrix()


# --- 4. Filter and Align Matrices ---
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
all_significant_cols <- union(sig_cols_anchor, sig_cols_neighbor)
final_col_order <- intersect(initial_col_order, all_significant_cols)

X_anchor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_anchor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
X_neighbor_aligned <- matrix(NA, nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))
Xsig_neighbor_aligned <- matrix("", nrow=length(final_row_order), ncol=length(final_col_order), dimnames=list(final_row_order, final_col_order))

common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)
X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_anchor_raw[common_rows_neighbor, common_cols_neighbor] #X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]


# --- 5. Create Combined Significance Matrix & Prepare Objects ---

Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""

Xsig_combined <- matrix("None",
                        nrow = nrow(Xsig_anchor_aligned),
                        ncol = ncol(Xsig_anchor_aligned),
                        dimnames = dimnames(Xsig_anchor_aligned))

Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned != "*"] <- "Anchor Only"
Xsig_combined[Xsig_anchor_aligned != "*" & Xsig_neighbor_aligned == "*"] <- "Neighbor Only"
Xsig_combined[Xsig_anchor_aligned == "*" & Xsig_neighbor_aligned == "*"] <- "Both"

rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_combined) <- dimnames(X_anchor_aligned)

rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)

names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order]
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")


# --- 6. Generate the Integrated Heatmap ---
star_colors <- c("Both" = "red", "Anchor Only" = "white", "Neighbor Only" = "black")

# --- UPDATED COLOR LOGIC START ---
# 1. Combine data to find the global range
all_values <- c(X_anchor_aligned, X_neighbor_aligned)

# 2. Find the maximum absolute value of the 5th/95th percentiles.
#    This creates a symmetric limit (e.g., if range is -2 to 5, limit becomes 5).
#    This ensures -limit and +limit are equidistant from 0.
limit <- max(abs(quantile(all_values, c(0.05, 0.95), na.rm = TRUE)))

# 3. Define the palette.
#    We use 11 colors (max for PuOr) to get the deepest purple and orange.
#    We keep your 'rev' logic (Purple=Low, Orange=High).
puor_colors <- rev(RColorBrewer::brewer.pal(8, "PuOr"))

# 4. Create the mapping with 3 fixed anchors: Min -> 0 (White) -> Max
col_fun <- colorRamp2(
  c(-limit, 0, limit),
  c(puor_colors[1], "white", puor_colors[8])
)
# --- UPDATED COLOR LOGIC END ---

ht_combined <- Heatmap(
  X_neighbor_aligned,
  name = "Colocalization\nlogFC",
  col = col_fun,
  cluster_rows = FALSE,
  cluster_columns = TRUE,
  cluster_column_slices = FALSE,
  show_column_dend = FALSE,
  row_split = row_slices_vec[rownames(X_neighbor_aligned)],
  column_split = column_slices_vec[colnames(X_neighbor_aligned)],
  rect_gp = gpar(col = "white", lwd = 1),
  row_names_gp = gpar(fontsize = 8),
  column_names_gp = gpar(fontsize = 7),
  row_title_gp = gpar(fontsize = 0),
  column_title_gp = gpar(fontsize = 0),
  heatmap_legend_param = list(
      title_gp = gpar(fontsize=7), 
      labels_gp = gpar(fontsize=7),
      direction = "horizontal" # MODIFIED
  ),
  top_annotation = HeatmapAnnotation(
    `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
    annotation_name_side = "left",
    annotation_name_gp = gpar(fontsize = 8)
  ),
  bottom_annotation = HeatmapAnnotation(
    show_legend = TRUE,
    Lineage = anndf_neighbors[colnames(X_neighbor_aligned), "Neighbors", drop = TRUE],
    annotation_name_gp = gpar(fontsize = 0),
    col = list(Lineage = lineage_palette),
    annotation_legend_param = list(
        Lineage = list(
            title_gp = gpar(fontsize=7), 
            labels_gp = gpar(fontsize=7),
            direction = "horizontal",
            ncol = 2 # MODIFIED: Arrange legend into 2 columns
        )
    ),
    simple_anno_size = unit(2, "mm")
  ),
  cell_fun = function(j, i, x, y, w, h, fill) {
    status <- Xsig_combined[i, j]
    if (status != "None") {
      grid.text("*", x, y, gp = gpar(col = star_colors[status], fontsize = 14))
    }
  }
)
# --- 7. Create a Custom Legend for the Stars ---

star_legend <- Legend(
  labels = c("Significant in Both", "Significant in T as Anchor Only", "Significant in T as Neighbor Only"),
  title = "Interaction Significance",
  legend_gp = gpar(col = c("red", "white", "black"), fontsize = 8),
  pch = "*",
  type = "points",
  labels_gp = gpar(fontsize = 8),
  title_gp = gpar(fontsize = 8, fontface = "plain"),
  direction = "horizontal" # MODIFIED
)


# --- 8. Draw the Final Plot ---
fig.size(height = 4.3, width = 6.3, res = 400)
draw(
  ht_combined,
  heatmap_legend_side = "bottom",
  annotation_legend_side = "bottom",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)

pdf('suppfig5A_merged_colors.pdf', width = 6.3, height = 4.3,  useKerning = TRUE)
draw(
  ht_combined,
  heatmap_legend_side = "bottom",
  annotation_legend_side = "bottom",
  annotation_legend_list = list(star_legend),
  merge_legends = TRUE # MODIFIED
)
dev.off()

# Other locations around the interface

# Around hub- interfaces [-40, 0]: T cells as anchors

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) %>%
        filter(cxcl_pos_tile == 'CXCL_neg') %>%
        filter(tessera_annotation == 'Stromal-enriched') %>%
        filter(closest_interface_type == 'CXCLneg tumor & CXCLneg stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

anchor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
neighbor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_hubNeg_T_as_anchor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Around hub- interfaces [-40, 0]: T cells as neighbors

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRd') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) %>%
        filter(cxcl_pos_tile == 'CXCL_neg') %>%
        filter(tessera_annotation == 'Stromal-enriched') %>%
        filter(closest_interface_type == 'CXCLneg tumor & CXCLneg stroma')

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

neighbor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
anchor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_hubNeg_T_as_neighbor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Combined heatmap

In [ ]:
anchor_list = c(fread("0_to_neg40_hubNeg_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor),
                fread("0_to_neg40_hubNeg_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(anchor)) %>% 
            unique

neighbor_list = c(fread("0_to_neg40_hubNeg_T_as_neighbor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor),
                fread("0_to_neg40_hubNeg_T_as_anchor_EPI_COLLAPSED.csv") %>% filter(padj < 0.05) %>% pull(neighbor)) %>% 
            unique

anchor_list %>% length
neighbor_list %>% length

In [ ]:
# Define the canonical order and splits for columns (all other cell types)
column_slices_vec <- anndf %>%
  mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
  pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
  mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
# Store the full, desired order of rows and columns. This will be filtered later.
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec) # Store the initial order before filtering
column_slices_vec
initial_col_order

In [ ]:
anndf_neighbors

In [ ]:
## -----------------------------------------------------------------------------
## Script: Aligned and Concatenated Interaction Heatmaps
##
## This script generates two heatmaps of cell-cell interaction logFC values,
## one for "T cells as anchor" and one for "T cells as neighbor". It ensures
## that both heatmaps are perfectly aligned by row and column, removes any
## columns that are empty in both datasets, and concatenates them vertically
## for a final comparative visualization.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

# Load necessary packages for data manipulation and visualization
library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer) # Added for the color palette


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)


# Create a vector of cell counts to use in the 'N Cells' annotation
cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRd",
   dist_interface_signed >= -40,
   dist_interface_signed <= 0,
   cxcl_pos_tile == "CXCL_neg",
   closest_interface_type == "CXCLneg tumor & CXCLneg stroma"
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 # Standardize cell type names by replacing underscores and shortening names
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

# Convert to a named vector for easy lookup
cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

# Create annotation data frames for the heatmap sidebars (cell lineages)
anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

# Define the canonical order and splits for rows (T-cell subtypes)
row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
# Define the canonical order and splits for columns (all other cell types)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
# Store the full, desired order of rows and columns. This will be filtered later.
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec) # Store the initial order before filtering


# --- 3. Process Raw Data Files ---

## 3.1 Load and clean "T as Anchor" data
summarized_res_anchor <- fread("0_to_neg40_hubNeg_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data from long to wide format for the heatmap matrix
X_anchor_raw <- dcast(
 summarized_res_anchor, anchor ~ neighbor,
 value.var = "logFC"
) %>%
 column_to_rownames("anchor") %>%
 as.matrix()

# Create a corresponding matrix for significance stars
Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>%
 as.matrix()

## 3.2 Load and clean "T as Neighbor" data
summarized_res_neighbor <- fread("0_to_neg40_hubNeg_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data (NOTE: formula is swapped to keep T cells as rows)
X_neighbor_raw <- dcast(
 summarized_res_neighbor, neighbor ~ anchor,
 value.var = "logFC"
) %>%
 column_to_rownames("neighbor") %>%
 as.matrix()

# Create the significance matrix for the neighbor data
Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.05 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>%
 as.matrix()

# --- 4. Filter and Align Matrices ---

## 4.1 Find and keep columns that have significant interactions in EITHER dataset
# Find column names with at least one star in the 'anchor' data
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
# Find column names with at least one star in the 'neighbor' data
sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
# Combine the two lists to get all columns that are significant in at least one heatmap
all_significant_cols <- union(sig_cols_anchor, sig_cols_neighbor)
# Filter your desired column order to only keep columns from the significant list
final_col_order <- intersect(initial_col_order, all_significant_cols)


## 4.2 Create perfectly aligned matrices using the filtered column order
# Create empty matrices with the final desired shape and order
X_anchor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_anchor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
X_neighbor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_neighbor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)

# Find common cells between raw and aligned matrices and populate the data
common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)

X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]

# Replace NA values with empty strings to prevent errors in the plotting function
Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""

# --- 5. Prepare Final Plotting Objects ---

# Apply string wrapping to all labels for consistent, clean plotting
rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(Xsig_anchor_aligned) <- dimnames(X_anchor_aligned)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_neighbor_aligned) <- dimnames(X_anchor_aligned)

# Update annotation data frames with wrapped labels
rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
anndf$Anchors <- str_wrap(anndf$Anchors, width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)
anndf_neighbors$Neighbors <- str_wrap(anndf_neighbors$Neighbors, width = 20)

# Update and filter split vectors with wrapped labels
names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order] # Filter to match new columns
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

# Format cell count numbers with commas for annotations
# Filter the original count vector to match the final row/column order
formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")


# --- 6. Generate Heatmaps ---

# Define common legend parameters for a consistent look
legend_params <- list(
 title_gp = gpar(fontsize = 7),
 labels_gp = gpar(fontsize = 7),
 direction = "vertical"
)

# Define a shared color function so the color scale is identical for both heatmaps
value_range <- quantile(c(X_anchor_aligned, X_neighbor_aligned), c(0.05, 0.95), na.rm = TRUE)
col_fun <- colorRamp2(
 seq(value_range[1], value_range[2], length.out = 10),
 rev(RColorBrewer::brewer.pal(10, "PiYG"))
)

## 6.1 Heatmap 1: T as Anchor
ht_anchor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_anchor_aligned,
 name = "Colocalization\nlogFC",
 col = col_fun,

 # Clustering and ordering
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_anchor_aligned)],
 column_split = column_slices_vec[colnames(X_anchor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),
 heatmap_legend_param = legend_params,

 # Annotations
 top_annotation = HeatmapAnnotation(
   `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
   annotation_name_side = "left",
   annotation_name_gp = gpar(fontsize = 8)
 ),
 right_annotation = rowAnnotation(
   show_legend = FALSE,
   Anchors = anndf[rownames(X_anchor_aligned), "Anchors", drop = TRUE],
   col = list(Anchors = lineage_palette),
   annotation_legend_param = list(Anchors = legend_params),
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),

 # Custom function to draw significance stars in cells
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_anchor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

## 6.2 Heatmap 2: T as Neighbor
ht_neighbor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_neighbor_aligned,
 name = "logFC_neighbor",
 col = col_fun,

 # Hide redundant elements for a cleaner concatenated plot
 show_heatmap_legend = FALSE,
 show_row_names = TRUE,
 show_column_names = TRUE,

 # Clustering and ordering (must be identical to the first heatmap)
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_neighbor_aligned)],
 column_split = column_slices_vec[colnames(X_neighbor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),

 # Annotations (with legends hidden)
 right_annotation = rowAnnotation(
   Neighbors = anndf[rownames(X_neighbor_aligned), "Anchors", drop = TRUE],
   col = list(Neighbors = lineage_palette),
   show_legend = FALSE,
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 bottom_annotation = HeatmapAnnotation(
   show_legend = TRUE,
   Lineage = anndf_neighbors[colnames(X_anchor_aligned), "Neighbors", drop = TRUE],
   annotation_name_gp = gpar(fontsize = 0),
   col = list(Lineage = lineage_palette),
   annotation_legend_param = list(Lineage = legend_params),
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 # Custom cell function for stars
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_neighbor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

# --- 7. Concatenate and Draw Final Plot ---

# Combine the two heatmaps vertically using the %v% operator
set.seed(42)
ht_list <- ht_anchor %v% ht_neighbor

fig.size(height = 5, width = 7, res = 400)
# Draw the final, combined plot
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)

pdf('hubNeg_coloc.pdf', width = 7, height = 5,  useKerning = TRUE)
# Draw the final, combined plot
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)
dev.off()

# MMRp interfaces

# Around MMRp interfaces [-40, 0]: T cells as anchors

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRp') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) %>%
        filter(tessera_annotation == 'Stromal-enriched') 

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

anchor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
neighbor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_MMRp_T_as_anchor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Around hub+ interfaces [-40, 0]: T cells as neighbors

In [ ]:
.temp = binned_cells_df %>%
        filter(MMRstatus == 'MMRp') %>%
        filter(dist_interface_signed > -40) %>%
        filter(dist_interface_signed < 0) #%>%
        #filter(tessera_annotation == 'Stromal-enriched') 

## Remove super rare populations
types_keep = names(which(table(.temp$type_lvl3) >= 100))

types_keep %>% length 
.temp = .temp[type_lvl3 %in% types_keep]

all_types = unique(.temp$type_lvl3)

neighbor_types = grep('Tcd8|Tcd4|Tpl', all_types, value = TRUE)
anchor_types = all_types
length(anchor_types)
length(neighbor_types)
length(all_types)

In [ ]:
options(future.globals.maxSize = 1e10)
res = calculate_spatial_enrichment(
    .temp,
    anchor_types,
    neighbor_types,
    max_dist = 30,
    nsteps = 3,
    use_multicore = TRUE
)
summarized_res = summarize_enrichment_results(res, n_total = 8, c_prior = 0.1)
fwrite(x = summarized_res, file = '0_to_neg40_MMRp_T_as_neighbor_EPI_COLLAPSED.csv')
glimpse(summarized_res)

# Combined heatmap

In [ ]:
anchor_list = c(fread("0_to_neg40_MMRp_T_as_neighbor_EPI_COLLAPSED.csv") %>% pull(anchor),
                fread("0_to_neg40_MMRp_T_as_anchor_EPI_COLLAPSED.csv") %>% pull(anchor)) %>% 
            unique

neighbor_list = c(fread("0_to_neg40_MMRp_T_as_neighbor_EPI_COLLAPSED.csv") %>% pull(neighbor),
                fread("0_to_neg40_MMRp_T_as_anchor_EPI_COLLAPSED.csv") %>% pull(neighbor)) %>% 
            unique

anchor_list %>% length
neighbor_list %>% length

In [ ]:
# Define the canonical order and splits for columns (all other cell types)
column_slices_vec <- anndf %>%
  mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
  pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
  mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
# Store the full, desired order of rows and columns. This will be filtered later.
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec) # Store the initial order before filtering
column_slices_vec
initial_col_order

In [ ]:
## -----------------------------------------------------------------------------
## Script: Aligned and Concatenated Interaction Heatmaps
##
## This script generates two heatmaps of cell-cell interaction logFC values,
## one for "T cells as anchor" and one for "T cells as neighbor". It ensures
## that both heatmaps are perfectly aligned by row and column, removes any
## columns that are empty in both datasets, and concatenates them vertically
## for a final comparative visualization.
## -----------------------------------------------------------------------------


# --- 1. Load Libraries and Setup ---

# Load necessary packages for data manipulation and visualization
library(data.table)
library(dplyr)
library(tidyr)
library(tibble)
library(stringr)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer) # Added for the color palette


# --- 2. Prepare Common Annotations, Order, and Palettes ---
lineage_palette <- c(
 'Epi' = '#CA49FC',
 'Strom' = '#00D2D0',
 'Myeloid' = '#FFB946',
 'Mast' = '#F4ED57',
 'Plasma' = '#61BDFC',
 'B' = '#0022FA',
 'TNKILC' = '#FF3420'
)


# Create a vector of cell counts to use in the 'N Cells' annotation
cell_counts <- binned_cells_df %>%
 filter(
   MMRstatus == "MMRp",
   dist_interface_signed >= -40,
   dist_interface_signed <= 0,
   tessera_annotation == 'Stromal-enriched'
 ) %>%
 group_by(type_lvl3) %>%
 summarize(n = n()) %>%
 # Standardize cell type names by replacing underscores and shortening names
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3))

cell_counts
# Convert to a named vector for easy lookup
cell_counts_vec <- cell_counts$n
names(cell_counts_vec) <- cell_counts$type_lvl3

# Create annotation data frames for the heatmap sidebars (cell lineages)
anndf <- binned_cells_df %>%
 select(type_lvl1, type_lvl3) %>%
 distinct() %>%
 as.data.frame() %>%
 mutate(type_lvl3 = gsub("_", "-", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Macro", "Macro", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-Mono", "Mono", type_lvl3)) %>%
 mutate(type_lvl3 = gsub("Myeloid-DC", "DC", type_lvl3)) %>%
 rename(Anchors = type_lvl1)

rownames(anndf) <- anndf$type_lvl3
anndf <- anndf %>% select(-type_lvl3)
anndf_neighbors <- anndf %>% rename(Neighbors = Anchors)

# Define the canonical order and splits for rows (T-cell subtypes)
row_slices_vec <- as.factor(c(rep("Tcd8", 6), rep("Tcd4", 4)))
names(row_slices_vec) <- c(
 "Tcd8-CXCL13", "Tcd8-HOBIT", "Tcd8-gdlike", "Tcd8-gdlike-PD1", "Tplzf-gdlike", "Tcd8-GZMK",
 "Tcd4-CXCL13","Tcd4-Treg", "Tcd4-IL7R", "Tcd4-TFH"
)
# Define the canonical order and splits for columns (all other cell types)
column_slices_vec <- anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)) %>%
 pull(Anchors)
names(column_slices_vec) <- rownames(anndf %>%
 mutate(Anchors = factor(Anchors, levels = c('Epi', 'Myeloid', 'Strom', 'TNKILC', 'B', 'Plasma', 'Mast'), ordered = TRUE)))
# Store the full, desired order of rows and columns. This will be filtered later.
final_row_order <- names(row_slices_vec)
initial_col_order <- names(column_slices_vec) # Store the initial order before filtering


# --- 3. Process Raw Data Files ---

## 3.1 Load and clean "T as Anchor" data
summarized_res_anchor <- fread("0_to_neg40_MMRp_T_as_anchor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data from long to wide format for the heatmap matrix
X_anchor_raw <- dcast(
 summarized_res_anchor, anchor ~ neighbor,
 value.var = "logFC"
) %>%
 column_to_rownames("anchor") %>%
 as.matrix()

# Create a corresponding matrix for significance stars
Xsig_anchor_raw <- summarized_res_anchor %>%
 mutate(txt = if_else(padj < 0.5 & logFC > 0, "*", "", "")) %>%
 dcast(anchor ~ neighbor, value.var = "txt") %>%
 column_to_rownames("anchor") %>%
 as.matrix()

## 3.2 Load and clean "T as Neighbor" data
summarized_res_neighbor <- fread("0_to_neg40_MMRp_T_as_neighbor_EPI_COLLAPSED.csv") %>%
 mutate(across(c(anchor, neighbor), ~ gsub("_", "-", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Macro", "Macro", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-Mono", "Mono", .x))) %>%
 mutate(across(c(anchor, neighbor), ~ gsub("Myeloid-DC", "DC", .x)))

# Reshape data (NOTE: formula is swapped to keep T cells as rows)
X_neighbor_raw <- dcast(
 summarized_res_neighbor, neighbor ~ anchor,
 value.var = "logFC"
) %>%
 column_to_rownames("neighbor") %>%
 as.matrix()

# Create the significance matrix for the neighbor data
Xsig_neighbor_raw <- summarized_res_neighbor %>%
 mutate(txt = if_else(padj < 0.5 & logFC > 0, "*", "", "")) %>%
 dcast(neighbor ~ anchor, value.var = "txt") %>%
 column_to_rownames("neighbor") %>%
 as.matrix()

# --- 4. Filter and Align Matrices ---

## 4.1 Find and keep columns that have significant interactions in EITHER dataset
# Find column names with at least one star in the 'anchor' data
sig_cols_anchor <- names(which(colSums(Xsig_anchor_raw == "*", na.rm = TRUE) > 0))
# Find column names with at least one star in the 'neighbor' data
sig_cols_neighbor <- names(which(colSums(Xsig_neighbor_raw == "*", na.rm = TRUE) > 0))
# Combine the two lists to get all columns that are significant in at least one heatmap
all_significant_cols <- union(sig_cols_anchor, sig_cols_neighbor)
# Filter your desired column order to only keep columns from the significant list
final_col_order <- intersect(initial_col_order, all_significant_cols)


## 4.2 Create perfectly aligned matrices using the filtered column order
# Create empty matrices with the final desired shape and order
X_anchor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_anchor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
X_neighbor_aligned <- matrix(NA,
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)
Xsig_neighbor_aligned <- matrix("",
 nrow = length(final_row_order), ncol = length(final_col_order),
 dimnames = list(final_row_order, final_col_order)
)

# Find common cells between raw and aligned matrices and populate the data
common_rows_anchor <- intersect(rownames(X_anchor_raw), final_row_order)
common_cols_anchor <- intersect(colnames(X_anchor_raw), final_col_order)

X_anchor_aligned[common_rows_anchor, common_cols_anchor] <- X_anchor_raw[common_rows_anchor, common_cols_anchor]
Xsig_anchor_aligned[common_rows_anchor, common_cols_anchor] <- Xsig_anchor_raw[common_rows_anchor, common_cols_anchor]

common_rows_neighbor <- intersect(rownames(X_neighbor_raw), final_row_order)
common_cols_neighbor <- intersect(colnames(X_neighbor_raw), final_col_order)
X_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- X_neighbor_raw[common_rows_neighbor, common_cols_neighbor]
Xsig_neighbor_aligned[common_rows_neighbor, common_cols_neighbor] <- Xsig_neighbor_raw[common_rows_neighbor, common_cols_neighbor]

# Replace NA values with empty strings to prevent errors in the plotting function
Xsig_anchor_aligned[is.na(Xsig_anchor_aligned)] <- ""
Xsig_neighbor_aligned[is.na(Xsig_neighbor_aligned)] <- ""

# --- 5. Prepare Final Plotting Objects ---

# Apply string wrapping to all labels for consistent, clean plotting
rownames(X_anchor_aligned) <- str_wrap(rownames(X_anchor_aligned), width = 20)
colnames(X_anchor_aligned) <- str_wrap(colnames(X_anchor_aligned), width = 20)
dimnames(Xsig_anchor_aligned) <- dimnames(X_anchor_aligned)
dimnames(X_neighbor_aligned) <- dimnames(X_anchor_aligned)
dimnames(Xsig_neighbor_aligned) <- dimnames(X_anchor_aligned)

# Update annotation data frames with wrapped labels
rownames(anndf) <- str_wrap(rownames(anndf), width = 20)
anndf$Anchors <- str_wrap(anndf$Anchors, width = 20)
rownames(anndf_neighbors) <- str_wrap(rownames(anndf_neighbors), width = 20)
anndf_neighbors$Neighbors <- str_wrap(anndf_neighbors$Neighbors, width = 20)

# Update and filter split vectors with wrapped labels
names(row_slices_vec) <- str_wrap(names(row_slices_vec), width = 20)
column_slices_vec <- column_slices_vec[final_col_order] # Filter to match new columns
names(column_slices_vec) <- str_wrap(names(column_slices_vec), width = 20)

# Format cell count numbers with commas for annotations
# Filter the original count vector to match the final row/column order
formatted_row_counts <- formatC(cell_counts_vec[final_row_order], big.mark = ",", format = "d")
formatted_col_counts <- formatC(cell_counts_vec[final_col_order], big.mark = ",", format = "d")

# --- 6. Generate Heatmaps ---

# Define common legend parameters for a consistent look
legend_params <- list(
 title_gp = gpar(fontsize = 7),
 labels_gp = gpar(fontsize = 7),
 direction = "vertical"
)

# Define a shared color function so the color scale is identical for both heatmaps
value_range <- quantile(c(X_anchor_aligned, X_neighbor_aligned), c(0.05, 0.95), na.rm = TRUE)
col_fun <- colorRamp2(
 seq(value_range[1], value_range[2], length.out = 10),
 rev(RColorBrewer::brewer.pal(10, "PiYG"))
)

## 6.1 Heatmap 1: T as Anchor
ht_anchor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_anchor_aligned,
 name = "Colocalization\nlogFC",
 col = col_fun,

 # Clustering and ordering
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_anchor_aligned)],
 column_split = column_slices_vec[colnames(X_anchor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),
 heatmap_legend_param = legend_params,

 # Annotations
 top_annotation = HeatmapAnnotation(
   `N Cells` = anno_text(formatted_col_counts, rot = 90, gp = gpar(fontsize = 7)),
   annotation_name_side = "left",
   annotation_name_gp = gpar(fontsize = 8)
 ),
 right_annotation = rowAnnotation(
   show_legend = FALSE,
   Anchors = anndf[rownames(X_anchor_aligned), "Anchors", drop = TRUE],
   col = list(Anchors = lineage_palette),
   annotation_legend_param = list(Anchors = legend_params),
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),

 # Custom function to draw significance stars in cells
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_anchor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

## 6.2 Heatmap 2: T as Neighbor
ht_neighbor <- Heatmap(gap = unit(0.25, "mm"), column_gap = unit(0.25, "mm"),
 X_neighbor_aligned,
 name = "logFC_neighbor",
 col = col_fun,

 # Hide redundant elements for a cleaner concatenated plot
 show_heatmap_legend = FALSE,
 show_row_names = TRUE,
 show_column_names = TRUE,

 # Clustering and ordering (must be identical to the first heatmap)
 cluster_rows = FALSE,
 cluster_columns = TRUE,
 cluster_column_slices = FALSE,
 show_column_dend = FALSE,
 row_split = row_slices_vec[rownames(X_neighbor_aligned)],
 column_split = column_slices_vec[colnames(X_neighbor_aligned)],

 # Aesthetics
 rect_gp = gpar(col = "white", lwd = 1),
 row_names_gp = gpar(fontsize = 7),
 column_names_gp = gpar(fontsize = 7),
 row_title_gp = gpar(fontsize = 0),
 column_title_gp = gpar(fontsize = 0),

 # Annotations (with legends hidden)
 right_annotation = rowAnnotation(
   Neighbors = anndf[rownames(X_neighbor_aligned), "Anchors", drop = TRUE],
   col = list(Neighbors = lineage_palette),
   show_legend = FALSE,
   annotation_name_side = "top",
   annotation_name_gp = gpar(fontsize = 10, fontface = 'bold'),
   annotation_name_rot = 0,
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 bottom_annotation = HeatmapAnnotation(
   show_legend = TRUE,
   Lineage = anndf_neighbors[colnames(X_anchor_aligned), "Neighbors", drop = TRUE],
   annotation_name_gp = gpar(fontsize = 0),
   col = list(Lineage = lineage_palette),
   annotation_legend_param = list(Lineage = legend_params),
   simple_anno_size = unit(2, "mm") # Controls annotation height
 ),
 # Custom cell function for stars
 cell_fun = function(j, i, x, y, w, h, fill) {
   if (Xsig_neighbor_aligned[i, j] == "*") {
     grid.text("*", x, y, gp = gpar(col = "white", fontsize = 8))
   }
 }
)

# --- 7. Concatenate and Draw Final Plot ---

# Combine the two heatmaps vertically using the %v% operator
set.seed(42)
ht_list <- ht_anchor %v% ht_neighbor

fig.size(height = 5, width = 7, res = 400)
# Draw the final, combined plot
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)

pdf('MMRp_coloc.pdf', width = 7, height = 5,  useKerning = TRUE)
# Draw the final, combined plot
draw(ht_list,
 heatmap_legend_side = "right",
 annotation_legend_side = "right",
 ht_gap = unit(5, "mm")
)
dev.off()